# Attributed C-Sets as Data Structure

This notebook is based on the paper
[Categorical Data Structures for Technical Computing](https://arxiv.org/pdf/2106.04703.pdf).

An Attributed C-Set, ACSet for short, is a data structure that aims to solve the
divide of combinatorial vs atomic data.

Data can be stored in many different formats, such as SQL tables, NO-SQL tables, data frames, and so on.
These different formats make difficult to analyze the data directly, since simple tasks, such as calculating the mean aggregating the data according to
an specific attribute, will require a different set of commands for each data format in order to be performed.
In order to avoid having to deal with each possible variation, most data analysis starts by turning the dataset into
data frame format.

The choice of the data frame as the centralizing data structure is understandable, since most of analysis
consists of data that can be thought of as single observations (rows) comprised of many features (columns).
Yet, there are many common scenarios where such data structure is not the most natural one. Perhaps the most
clear example are graphs. Here, the main aspect of the data is not it's "atomic" nature, but it's relational
information ("which nodes are connected").

The use o relational databases (SQL) can deal with such divide, but are usually too stiff, since they usually
are part of a monolithic system with it's own langugae, which is not always straightfoward to integrate
with general purpose programming languages such as Julia.

ACSets were the solution proposed by Evan Patterson, Owen Lynch, and James Fairbanks.
It consists of an efficient in-memory implementation of categorical databases, which encompasses data structures
such as data frames, graphs and more. Thus, solving the combinatorial vs atomic data representation problem.

In the implementation of ACSets, combinatorial data is always represented by integers, while atomic data
is represented by type parameters which can be Julia types.

In [1]:
using Pkg
Pkg.activate(".")
using Catlab,Catlab.CategoricalAlgebra, Catlab.ACSetInterface, Catlab.Graphs
using Catlab.Graphics
using Catlab.Graphics.Graphviz
using Catlab.Programs.DiagrammaticPrograms
using Colors
draw(g) = to_graphviz(g, node_labels=true, edge_labels=true)

  Activating project at `~/Main/EMAp/Mathematical-Short-Notes/Fields/Category-Theory/notebooks`


draw (generic function with 1 method)

## Example 1 - RoadMap

This first example is directly from the paper. The idea here is to create an Acset to store
information about roads, where `vertices` are intersctions, `edges` are the roads from each intersection,
and `lenght` is the actual distance between interesection.

In [2]:
@present TheoryRoadMap(FreeSchema) begin
    (V,E)::Ob
    (src,tgt)::Hom(E,V)
    T::AttrType
    (x,y)::Attr(V,T)
    length::Attr(E,T)
end

@acset_type RoadMap(TheoryRoadMap, index=[:src,:tgt])

RoadMap

First we define the data schema, i.e. what are our tables and their relations. This is similar to what we have
in relational databases, but in the categorical form. In our example, we want to have two tables,
one is a list of vertices with their coordinates, and the other is the list of edges, where we
have the source and target vertices of each edge, and the length.

Hence, `(V,E)::Ob` states that we have two objects (tables) `V` and `E`.
The line `T::AttrType` indicates that our data has values with type `T`.

Next, `(x,y)::Attr(V,T)` indicates that we have two morphisms, i.e. `x:V -> T` and `y:V ->T`.
And `length::Attrb(E,T)` is a morphism `length` from edges to type `T`.

In [3]:
function make_path(coords::Vector{Tuple{Float64, Float64}})
    # Create an empty roadmap
    path = RoadMap{Float64}()
    # This is a convenient function that calculates the Euclidean distance between two
    # vertices in the road map. Notice that we can reference attributes using indexing
    # and that the system knows that these attributes belong to vertices, not edges.
    dist(i,j) = sqrt((path[i,:x] - path[j,:x])^2 + (path[i,:y] - path[j,:y])^2)
    x, y = coords[1]
    # add_part! mutates path to add a part, returning the index of the added part.
    # The named arguments to this function assign the attributes of that part.
    src = add_part!(path, :V, x=x, y=y)
    for i in 2:length(coords)
        x, y = coords[i]
        tgt = add_part!(path, :V, x=x, y=y)
        add_part!(path, :E, src=src, tgt=tgt, length=dist(src,tgt))
        src = tgt
    end
    path
end
    
ac = make_path([(x[1],x[2]) for x in eachrow(rand(10,2))])

V,x,y
1,0.452075,0.913903
2,0.873634,0.265936
3,0.217563,0.812025
4,0.752655,0.930724
5,0.00751398,0.0415869
6,0.0870555,0.719029
7,0.590724,0.62993
8,0.281664,0.719091
9,0.447633,0.792288
10,0.864583,0.717466


In [4]:
subpart(ac, 2, [:src, :x]) # Get source vertex from edge 2 and take :x attribute.

0.873634450122913

In [5]:
ACSetInterface.tables(ac)

(V = Catlab.CSetDataStructures.StructACSetTable{RoadMap{Float64}, :V} with 10 rows, 2 columns, and an unknown schema., E = Catlab.CSetDataStructures.StructACSetTable{RoadMap{Float64}, :E} with 9 rows, 3 columns, and an unknown schema.)

In [6]:
ac.homs.tgt

9-element Vector{Int64}:
  2
  3
  4
  5
  6
  7
  8
  9
 10

Let's change a bit this example. Instead of only a single Type attribute, let's use two.

In [7]:
@present TheoryMeshMap(FreeSchema) begin
    (V,E)::Ob
    (src,tgt)::Hom(E,V)
    (S,T)::AttrType
    (x,y)::Attr(V,T)
    length::Attr(E,S)
end

@acset_type MeshMap(TheoryMeshMap, index=[:src,:tgt])

MeshMap

Now, let's instantiate this ACSet. 

In [8]:
path = MeshMap{Float64, Int}()

MeshMap{Float64, Int64} with elements V = 1:0, E = 1:0

## Example 2 - Decorated Graphs

Let's enrich our graph structure to allow vertices and edges attributes.

In [9]:
using Catlab.CategoricalAlgebra
@present TheoryDecoratedGraph(FreeSchema) begin
    (E,V)::Ob
    (src,tgt)::Hom(E,V)
    (DotType,Shape)::AttrType
    dots::Attr(E,DotType)
    shape::Attr(V,Shape)
end

Presentation{Catlab.Theories.Schema, Symbol}(Catlab.Theories.FreeSchema, (Ob = Catlab.Theories.FreeSchema.Ob{:generator}[E, V], Hom = Catlab.Theories.FreeSchema.Hom{:generator}[src, tgt], AttrType = Catlab.Theories.FreeSchema.AttrType{:generator}[DotType, Shape], Attr = Catlab.Theories.FreeSchema.Attr{:generator}[dots, shape]), Dict(:shape => (:Attr => 2), :src => (:Hom => 1), :dots => (:Attr => 1), :V => (:Ob => 2), :DotType => (:AttrType => 1), :E => (:Ob => 1), :tgt => (:Hom => 2), :Shape => (:AttrType => 2)), Pair[])

In [10]:
@acset_type DecoratedGraph(TheoryDecoratedGraph, index=[:src, :tgt]);

In [11]:
F = @acset DecoratedGraph{Bool, Symbol} begin
    E = 5
    V = 4
    src   = [1,1,1,3,2]
    tgt   = [2,3,4,4,4]
    dots  = [true,false,false,false,true]
    shape = [:star,:square,:star,:circle]
end

DecoratedGraph{Bool, Symbol} with elements E = 1:5, V = 1:4
┌───┬─────┬─────┬───────┐
│ E │ src │ tgt │  dots │
├───┼─────┼─────┼───────┤
│ 1 │   1 │   2 │  true │
│ 2 │   1 │   3 │ false │
│ 3 │   1 │   4 │ false │
│ 4 │   3 │   4 │ false │
│ 5 │   2 │   4 │  true │
└───┴─────┴─────┴───────┘
┌───┬────────┐
│ V │  shape │
├───┼────────┤
│ 1 │   star │
│ 2 │ square │
│ 3 │   star │
│ 4 │ circle │
└───┴────────┘

In [12]:
add_part!(F, :E, src=4, tgt=1, dots=false);

In [13]:
F

DecoratedGraph{Bool, Symbol} with elements E = 1:6, V = 1:4
┌───┬─────┬─────┬───────┐
│ E │ src │ tgt │  dots │
├───┼─────┼─────┼───────┤
│ 1 │   1 │   2 │  true │
│ 2 │   1 │   3 │ false │
│ 3 │   1 │   4 │ false │
│ 4 │   3 │   4 │ false │
│ 5 │   2 │   4 │  true │
│ 6 │   4 │   1 │ false │
└───┴─────┴─────┴───────┘
┌───┬────────┐
│ V │  shape │
├───┼────────┤
│ 1 │   star │
│ 2 │ square │
│ 3 │   star │
│ 4 │ circle │
└───┴────────┘

Let's play a little with this data structure. 

In [14]:
@show incident(F, 4, :tgt)
@show incident(F, :star, :shape)
;

incident(F, 4, :tgt) = [3, 4, 5]
incident(F, :star, :shape) = [1, 3]


In [15]:
F.obs

2-element StaticArrays.MVector{2, Int64} with indices SOneTo(2):
 6
 4

## More complex examples

In [16]:
@present PASchema(FreeSchema) begin
    (Authors,Papers, Authorship)::Ob
    (p)::Hom(Authorship,Papers)
    (a)::Hom(Authorship,Authors)
    (T,N)::AttrType
    name::Attr(Authors,N)
    title::Attr(Papers,N)
    year::Attr(Papers,T)
end

@acset_type APA(PASchema, index=[:p,:a])

APA

In [17]:
ac = @acset APA{Real,String} begin
    Authors = 2
    Papers = 2
    Authorship = 3
    p = [1,2,2]
    a = [1,1,2]
    name = ["A","B"]
    title = ["Paper1","Paper2"]
    year  = [2000,2001]
end

APA{Real, String} with elements Authors = 1:2, Papers = 1:2, Authorship = 1:3
┌─────────┬──────┐
│ Authors │ name │
├─────────┼──────┤
│       1 │    A │
│       2 │    B │
└─────────┴──────┘
┌────────┬────────┬──────┐
│ Papers │  title │ year │
├────────┼────────┼──────┤
│      1 │ Paper1 │ 2000 │
│      2 │ Paper2 │ 2001 │
└────────┴────────┴──────┘
┌────────────┬───┬───┐
│ Authorship │ p │ a │
├────────────┼───┼───┤
│          1 │ 1 │ 1 │
│          2 │ 2 │ 1 │
│          3 │ 2 │ 2 │
└────────────┴───┴───┘

In [18]:
ac.attrs

(name = ["A", "B"], title = ["Paper1", "Paper2"], year = Real[2000, 2001])

In [19]:
typeof(ac) <: ACSet

true

## Example of Data Migration

In [20]:
@present SchGraph(FreeSchema) begin
  V::Ob
  E::Ob
  src::Hom(E,V)
  tgt::Hom(E,V)
end

@present SchPortGraph(FreeSchema) begin
  (Wire,Box,OutPort, InPort)::Ob
  src::Hom(Wire,OutPort)
  tgt::Hom(Wire,InPort)
  boxout::Hom(OutPort,Box)
  boxin::Hom(InPort,Box)
end

F = @migration SchGraph SchPortGraph begin
    V => Box
    E => Wire
    src => src ⋅ boxout
    tgt => tgt ⋅ boxin
end

FinFunctor(Dict{Symbol, Catlab.Theories.FreeSchema.Ob{:generator}}(:V => Box, :E => Wire), Dict{Symbol, Catlab.Theories.FreeSchema.Hom{:compose}}(:src => compose(src,boxout), :tgt => compose(tgt,boxin)), FinCat(Presentation{Schema, Symbol}(Catlab.Theories.FreeSchema, (Ob = Catlab.Theories.FreeSchema.Ob{:generator}[V, E], Hom = Catlab.Theories.FreeSchema.Hom{:generator}[src, tgt], AttrType = Catlab.Theories.FreeSchema.AttrType{:generator}[], Attr = Catlab.Theories.FreeSchema.Attr{:generator}[]), Dict(:src=>(:Hom=>1), :V=>(:Ob=>1), :E=>(:Ob=>2), :tgt=>(:Hom=>2)), Pair[])), FinCat(Presentation{Schema, Symbol}(Catlab.Theories.FreeSchema, (Ob = Catlab.Theories.FreeSchema.Ob{:generator}[Wire, Box, OutPort, InPort], Hom = Catlab.Theories.FreeSchema.Hom{:generator}[src, tgt, boxout, boxin], AttrType = Catlab.Theories.FreeSchema.AttrType{:generator}[], Attr = Catlab.Theories.FreeSchema.Attr{:generator}[]), Dict(:boxout=>(:Hom=>3), :Wire=>(:Ob=>1), :InPort=>(:Ob=>4), :src=>(:Hom=>1), :boxin=>(:H

In [24]:
g = star_graph(Graphs.Graph, 5)
h = @migrate SymmetricGraph g begin
  V => V
  E => @cases (fwd::E; rev::E)
  src => (fwd => src; rev => tgt)
  tgt => (fwd => tgt; rev => src)
  inv => (fwd => rev; rev => fwd)
end

E,src,tgt,inv
1,5,1,5
2,5,2,6
3,5,3,7
4,5,4,8
5,1,5,1
6,2,5,2
7,3,5,3
8,4,5,4


## Nested Acset

In [28]:
@present MySchema(FreeSchema) begin
    D::Ob
    (T1,T2,T3)::AttrType
    x::Attr(D,T1)
    y::Attr(D,T2)
    z::Attr(D,T3)
end

@acset_type MyAcset(MySchema)

df = @acset MyAcset{String, Int,Float64}  begin
    D = 10
    x = rand(["A","B","C"],10)
    y = rand([0,1],10)
    z = rand(10)
end

D,x,y,z
1,A,1,0.338882
2,C,1,0.381757
3,A,0,0.294181
4,A,0,0.385299
5,A,0,0.399548
6,B,1,0.48944
7,A,1,0.314148
8,C,0,0.168701
9,B,1,0.574559
10,A,1,0.566838


In [29]:
@present MyNestedSchema(FreeSchema) begin
    (X, Y, Z)::Ob
    y_::Hom(X, Y)
    z_::Hom(Y, Z)
    (T1,T2,T3)::AttrType
    x::Attr(X,T1)
    y::Attr(Y,T2)
    z::Attr(Z,T3)
end

@acset_type MyNestedAcset(MyNestedSchema)

MyNestedAcset

In [30]:
nf = @acset MyNestedAcset{String, Int,Float64}  begin
    X = 1
    Y = 1
    Z = 1
    x = ["A"]
    y = 1
    z = rand()
end

MyNestedAcset{String, Int64, Float64} with elements X = 1:1, Y = 1:1, Z = 1:1
┌───┬────┬───┐
│ X │ y_ │ x │
├───┼────┼───┤
│ 1 │  0 │ A │
└───┴────┴───┘
┌───┬────┬───┐
│ Y │ z_ │ y │
├───┼────┼───┤
│ 1 │  0 │ 1 │
└───┴────┴───┘
┌───┬──────────┐
│ Z │        z │
├───┼──────────┤
│ 1 │ 0.372211 │
└───┴──────────┘

In [40]:
typeof(df) <: ACSet

true

In [41]:
?migrate

search: migrate migrate! @migrate @migration DataMigration DataMigrations



Contravariantly migrate data from the acset `Y` to a new acset of type `T`.

The mutating variant of this function is [`migrate!`](@ref).


In [33]:
F = @migration MySchema MyNestedSchema begin
    D => X
    x => x
    y => y
    z => z
    T1 => T1
    T2 => T2
    T3 => T3
end

FinFunctor(Dict{Symbol, GATExpr{:generator}}(:T2 => T2, :D => X, :T1 => T1, :T3 => T3), Dict{Symbol, Catlab.Theories.FreeSchema.Attr{:generator}}(:y => y, :z => z, :x => x), FinCat(Presentation{Schema, Symbol}(Catlab.Theories.FreeSchema, (Ob = Catlab.Theories.FreeSchema.Ob{:generator}[D], Hom = Catlab.Theories.FreeSchema.Hom{:generator}[], AttrType = Catlab.Theories.FreeSchema.AttrType{:generator}[T1, T2, T3], Attr = Catlab.Theories.FreeSchema.Attr{:generator}[x, y, z]), Dict(:T2=>(:AttrType=>2), :D=>(:Ob=>1), :y=>(:Attr=>2), :z=>(:Attr=>3), :T1=>(:AttrType=>1), :T3=>(:AttrType=>3), :x=>(:Attr=>1)), Pair[])), FinCat(Presentation{Schema, Symbol}(Catlab.Theories.FreeSchema, (Ob = Catlab.Theories.FreeSchema.Ob{:generator}[X, Y, Z], Hom = Catlab.Theories.FreeSchema.Hom{:generator}[y_, z_], AttrType = Catlab.Theories.FreeSchema.AttrType{:generator}[T1, T2, T3], Attr = Catlab.Theories.FreeSchema.Attr{:generator}[x, y, z]), Dict(:T3=>(:AttrType=>3), :z_=>(:Hom=>2), :T2=>(:AttrType=>2), :X=>(:

In [34]:
migrate(df, F)

LoadError: MethodError: no method matching migrate(::Catlab.CategoricalAlgebra.CSets.ACSetFunctor{MyAcset{String, Int64, Float64}}, ::Catlab.CategoricalAlgebra.FinCats.FinDomFunctorMap{Catlab.CategoricalAlgebra.FinCats.FinCatPresentation{Catlab.Theories.Schema, Union{Catlab.Theories.FreeSchema.AttrType, Catlab.Theories.FreeSchema.Ob}, Union{Catlab.Theories.FreeSchema.Attr, Catlab.Theories.FreeSchema.AttrType, Catlab.Theories.FreeSchema.Hom}}, Catlab.CategoricalAlgebra.FinCats.FinCatPresentation{Catlab.Theories.Schema, Union{Catlab.Theories.FreeSchema.AttrType, Catlab.Theories.FreeSchema.Ob}, Union{Catlab.Theories.FreeSchema.Attr, Catlab.Theories.FreeSchema.AttrType, Catlab.Theories.FreeSchema.Hom}}, Dict{Symbol, GATExpr{:generator}}, Dict{Symbol, Catlab.Theories.FreeSchema.Attr{:generator}}})
[0mClosest candidates are:
[0m  migrate([91m::ACSet[39m, ::Functor{Dom} where Dom<:FinCat; kw...) at ~/.julia/packages/Catlab/XsT24/src/categorical_algebra/DataMigrations.jl:113
[0m  migrate(

In [ ]:

ac = @acset APA{Real,String} begin
    Authors = 2
    Papers = 2
    Authorship = 3
    p = [1,2,2]
    a = [1,1,2]
    name = ["A","B"]
    title = ["Paper1","Paper2"]
    year  = [2000,2001]
end

In [ ]:
# Write down the schema for a weighted graph
@present TheoryWeightedGraph(FreeSchema) begin
  V::Ob
  E::Ob
  src::Hom(E,V)
  tgt::Hom(E,V)
  T::AttrType
  weight::Attr(E,T)
end

# Construct the type used to store acsets on the previous schema
# We *index* src and tgt, which means that we store not only
# the forwards map, but also the backwards map.
@acset_type WeightedGraph(TheoryWeightedGraph, index=[:src,:tgt])

# Construct a weighted graph, with floats as edge weights
g = @acset WeightedGraph{Float64} begin
  V = 4
  E = 5
  src = [1,1,1,2,3]
  tgt = [2,3,4,4,4]
  weight = [7.2, 9.3, 9.4, 0.1, 42.0]
end

In [ ]:
# ϕ = ACSetTransformation(e,w,E=[1], V=[1,2])
